# Name: Sandipto Sanyal
# PGID: 12010004

# Install findspark

In [1]:
#!pip install findspark

# Import necessary libraries

In [2]:
import findspark
findspark.init()
import os
from pyspark import SparkContext
import csv
from StringIO import StringIO
import numpy as np

# Create SparkContext

In [3]:
sc = SparkContext(master="local", appName="Question 1")

# Define a function to read the csv and split the lines

In [4]:
def split(line):
    """
    Operator function for splitting a line with csv module
    """
    reader = csv.reader(StringIO(line))
    return reader.next()

# Question 1

## Create RDDs of the input

In [5]:
users_link = r'file:///home/cloudera/Desktop/git/big_data_management1/BDM 1 Assignment-20200716/users.csv'
transactions_link = r'file:///home/cloudera/Desktop/git/big_data_management1/BDM 1 Assignment-20200716/transactions.csv'
users_rdd = sc.textFile(users_link).cache()
transactions_rdd = sc.textFile(transactions_link).cache()

## Make transformations of the data
Below we make some transformations of the dataset so that it renders into a CSV format.

In [6]:
users_rdd = users_rdd.map(split)
transactions_rdd = transactions_rdd.map(split)

In [7]:
users_rdd.collect()

[['1', 'user1@company.com', 'ES', 'MX'],
 ['2', 'user4@domain.com', 'EN', 'US'],
 ['3', 'user5@company.com', 'FR', 'FR'],
 ['4', 'user9@site.org', 'HI', 'IN'],
 ['5', 'user12@service.io', 'EN', 'CA'],
 ['6', 'user17@website.net', 'FR', 'FR'],
 ['7', 'user21@company.com', 'FR', 'FR'],
 ['8', 'user25@company.com', 'FR', 'FR'],
 ['9', 'user27@school.edu', 'ES', 'MX'],
 ['10', 'user31@website.net', 'EN', 'CA'],
 ['11', 'user36@website.net', 'FR', 'FR'],
 ['12', 'user39@domain.com', 'FR', 'FR'],
 ['13', 'user41@company.com', 'ES', 'MX'],
 ['14', 'user45@domain.com', 'HI', 'IN'],
 ['15', 'user48@site.org', 'ES', 'MX'],
 ['16', 'user53@school.edu', 'EN', 'US'],
 ['17', 'user57@school.edu', 'ES', 'MX'],
 ['18', 'user59@website.net', 'HI', 'IN'],
 ['19', 'user64@school.edu', 'EN', 'US'],
 ['20', 'user67@domain.com', 'HI', 'IN'],
 ['21', 'user68@site.org', 'EN', 'US'],
 ['22', 'user71@domain.com', 'ES', 'MX'],
 ['23', 'user74@service.io', 'EN', 'US'],
 ['24', 'user79@website.net', 'ES', 'MX'],
 

In [8]:
transactions_rdd.collect()

[['1', '1004', '19', '129', 'whatchamacallit'],
 ['2', '1001', '10', '99', 'thingamajig'],
 ['3', '1004', '17', '129', 'whatchamacallit'],
 ['4', '1001', '9', '99', 'thingamajig'],
 ['5', '1003', '3', '89', 'gadget'],
 ['6', '1002', '19', '149', 'gizmo'],
 ['7', '1002', '30', '149', 'gizmo'],
 ['8', '1002', '26', '149', 'gizmo'],
 ['9', '1001', '22', '99', 'thingamajig'],
 ['10', '1003', '6', '89', 'gadget'],
 ['11', '1004', '1', '129', 'whatchamacallit'],
 ['12', '1004', '2', '129', 'whatchamacallit'],
 ['13', '1005', '5', '199', 'doohickey'],
 ['14', '1004', '7', '129', 'whatchamacallit'],
 ['15', '1002', '16', '149', 'gizmo']]

## a) Count of unique locations where each product is sold

### Create RDDs userid|location and userid|product to perform join on userid

In [9]:
def create_userid_location_rdd(row):
    data = [row[0], row[3]]
    return data
def create_userid_product_rdd(row):
    data = [row[2],row[4], float(row[3])]
    return data
userid_location_rdd = users_rdd.map(create_userid_location_rdd)
userid_product_rdd = transactions_rdd.map(create_userid_product_rdd)

In [10]:
joined_rdd = userid_location_rdd.rightOuterJoin(userid_product_rdd)

In [11]:
joined_rdd.collect()

[('26', ('IN', 'gizmo')),
 ('17', ('MX', 'whatchamacallit')),
 ('22', ('MX', 'thingamajig')),
 ('19', ('US', 'whatchamacallit')),
 ('19', ('US', 'gizmo')),
 ('1', ('MX', 'whatchamacallit')),
 ('3', ('FR', 'gadget')),
 ('5', ('CA', 'doohickey')),
 ('7', ('FR', 'whatchamacallit')),
 ('9', ('MX', 'thingamajig')),
 ('30', ('US', 'gizmo')),
 ('16', ('US', 'gizmo')),
 ('2', ('US', 'whatchamacallit')),
 ('6', ('FR', 'gadget')),
 ('10', ('CA', 'thingamajig'))]

In [12]:
grouped_rdd = joined_rdd.map(lambda row:(row[1][1],row[1][0])).groupByKey().collect()
grouped_rdd

[('thingamajig', <pyspark.resultiterable.ResultIterable at 0x7fa2f267c5d0>),
 ('gadget', <pyspark.resultiterable.ResultIterable at 0x7fa2f267c7d0>),
 ('doohickey', <pyspark.resultiterable.ResultIterable at 0x7fa2f267c710>),
 ('whatchamacallit',
  <pyspark.resultiterable.ResultIterable at 0x7fa2f267c790>),
 ('gizmo', <pyspark.resultiterable.ResultIterable at 0x7fa2f32b0890>)]

In [13]:
print("Count of unique locations where each product is sold:")
[(items, (np.unique(np.array((list(locations))))).shape[0]) for (items, locations) in grouped_rdd]

Count of unique locations where each product is sold:


[('thingamajig', 2),
 ('gadget', 1),
 ('doohickey', 1),
 ('whatchamacallit', 3),
 ('gizmo', 2)]

## b) Products bought by each user
To perform this we will make use of the joined_rdd that we created earlier in step a

In [14]:
grouped_rdd = joined_rdd.map(lambda row: (row[0],row[1][1])).groupByKey()
print('Products bought by each user:')
([(userid, list(items)) for (userid, items) in grouped_rdd.collect()])

Products bought by each user:


[('26', ['gizmo']),
 ('17', ['whatchamacallit']),
 ('22', ['thingamajig']),
 ('19', ['whatchamacallit', 'gizmo']),
 ('1', ['whatchamacallit']),
 ('3', ['gadget']),
 ('5', ['doohickey']),
 ('7', ['whatchamacallit']),
 ('9', ['thingamajig']),
 ('10', ['thingamajig']),
 ('30', ['gizmo']),
 ('2', ['whatchamacallit']),
 ('16', ['gizmo']),
 ('6', ['gadget'])]

# c) Total spending done by each user on each product
To perform this we take help of the userid_product_rdd

In [15]:
userid_product_rdd.collect()

[['19', 'whatchamacallit', 129.0],
 ['10', 'thingamajig', 99.0],
 ['17', 'whatchamacallit', 129.0],
 ['9', 'thingamajig', 99.0],
 ['3', 'gadget', 89.0],
 ['19', 'gizmo', 149.0],
 ['30', 'gizmo', 149.0],
 ['26', 'gizmo', 149.0],
 ['22', 'thingamajig', 99.0],
 ['6', 'gadget', 89.0],
 ['1', 'whatchamacallit', 129.0],
 ['2', 'whatchamacallit', 129.0],
 ['5', 'doohickey', 199.0],
 ['7', 'whatchamacallit', 129.0],
 ['16', 'gizmo', 149.0]]

In [16]:
spending_user_item = userid_product_rdd.map(lambda row: ((row[0],row[1]),row[2])).groupByKey().sortByKey()
print('Total spending done on each product by each user::')
([(key[0],key[1],sum(list(value))) for (key, value) in spending_user_item.collect()])

Total spending done on each product by each user::


[('1', 'whatchamacallit', 129.0),
 ('10', 'thingamajig', 99.0),
 ('16', 'gizmo', 149.0),
 ('17', 'whatchamacallit', 129.0),
 ('19', 'gizmo', 149.0),
 ('19', 'whatchamacallit', 129.0),
 ('2', 'whatchamacallit', 129.0),
 ('22', 'thingamajig', 99.0),
 ('26', 'gizmo', 149.0),
 ('3', 'gadget', 89.0),
 ('30', 'gizmo', 149.0),
 ('5', 'doohickey', 199.0),
 ('6', 'gadget', 89.0),
 ('7', 'whatchamacallit', 129.0),
 ('9', 'thingamajig', 99.0)]

# Question 2

## Read the CSV file and parse into new separated lines

In [17]:
olympics_link = r'file:///home/cloudera/Desktop/git/big_data_management1/BDM 1 Assignment-20200716/olympics.csv'
olympics_rdd = sc.textFile(olympics_link, use_unicode=False).cache()
olympics_rdd = olympics_rdd.map(split).map(lambda row: row[0].split('\t'))

In [18]:
olympics_rdd.first()

['Michael Phelps',
 '23',
 'United States',
 '2008',
 '08-24-08',
 'Swimming',
 '8',
 '0',
 '0',
 '8']

<font color='red'>Assuming columns are: player name, age, country, year, date, category, gold, silver, bronze, total

## a) Total medals that each country won in particular sport

In [19]:
total_medals = olympics_rdd.map(lambda row: [(row[2],row[5]), int(row[9])]).groupByKey().sortByKey().collect()
print('Total medals by country by sport')
([(key[0], key[1], sum(list(value))) for (key, value) in total_medals])

Total medals by country by sport


[('Afghanistan', 'Taekwondo', 2),
 ('Algeria', 'Athletics', 5),
 ('Algeria', 'Boxing', 1),
 ('Algeria', 'Judo', 2),
 ('Argentina', 'Basketball', 24),
 ('Argentina', 'Cycling', 2),
 ('Argentina', 'Football', 34),
 ('Argentina', 'Hockey', 65),
 ('Argentina', 'Judo', 1),
 ('Argentina', 'Sailing', 10),
 ('Argentina', 'Swimming', 1),
 ('Argentina', 'Taekwondo', 1),
 ('Argentina', 'Tennis', 3),
 ('Armenia', 'Boxing', 1),
 ('Armenia', 'Weightlifting', 5),
 ('Armenia', 'Wrestling', 4),
 ('Australia', 'Archery', 2),
 ('Australia', 'Athletics', 16),
 ('Australia', 'Baseball', 24),
 ('Australia', 'Basketball', 48),
 ('Australia', 'Beach Volleyball', 2),
 ('Australia', 'Canoeing', 19),
 ('Australia', 'Cycling', 36),
 ('Australia', 'Diving', 17),
 ('Australia', 'Equestrian', 10),
 ('Australia', 'Freestyle Skiing', 5),
 ('Australia', 'Hockey', 81),
 ('Australia', 'Judo', 1),
 ('Australia', 'Rowing', 61),
 ('Australia', 'Sailing', 21),
 ('Australia', 'Shooting', 6),
 ('Australia', 'Short-Track Speed 

# b) India medals in each olympic game

In [20]:
india_medals = olympics_rdd.filter(lambda row: row[2]=='India').map(lambda row:[(row[2],row[3]), int(row[9])]).groupByKey().sortByKey().collect()
print('Number of India medals in each olympic')
[(key[0], key[1], sum(list(value))) for (key, value) in india_medals]

Number of India medals in each olympic


[('India', '2000', 1),
 ('India', '2004', 1),
 ('India', '2008', 3),
 ('India', '2012', 6)]

# c) Top 3 countries by total medal by each olympic games

In [21]:
# first group by year country and total medals tally
country_year_medals_rdd = olympics_rdd.map(lambda row:[(row[3],row[2]),int(row[9])]).groupByKey().map(lambda row: (row[0][0],sum(list(row[1])),row[0][1]))
# sort the RDD based on year and total medals
country_year_medals_rdd = country_year_medals_rdd.sortBy(lambda row: (row[0],row[1]),ascending=False)
# group the RDD based on year
country_year_medals_rdd = country_year_medals_rdd.groupBy(lambda row: row[0]).map(lambda row: (row[0],list(row[1])[0:3]))
country_year_medals_rdd.collect()

[('2002',
  [('2002', 84, 'United States'),
   ('2002', 74, 'Canada'),
   ('2002', 61, 'Germany')]),
 ('2000',
  [('2000', 243, 'United States'),
   ('2000', 187, 'Russia'),
   ('2000', 183, 'Australia')]),
 ('2006',
  [('2006', 69, 'Canada'), ('2006', 64, 'Sweden'), ('2006', 54, 'Germany')]),
 ('2004',
  [('2004', 265, 'United States'),
   ('2004', 191, 'Russia'),
   ('2004', 156, 'Australia')]),
 ('2008',
  [('2008', 317, 'United States'),
   ('2008', 184, 'China'),
   ('2008', 149, 'Australia')]),
 ('2010',
  [('2010', 97, 'United States'),
   ('2010', 90, 'Canada'),
   ('2010', 54, 'Germany')]),
 ('2012',
  [('2012', 254, 'United States'),
   ('2012', 140, 'Russia'),
   ('2012', 126, 'Great Britain')])]

In [22]:
sc.stop()